In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from tqdm.auto import tqdm

In [2]:
# use cdf of normal distribution to get P(Z > 4)
a = 1 - norm.cdf(4, loc=0, scale=1)
print(a) # ground truth

3.167124183311998e-05


In [3]:
n = int(1e3)
m = 100000
experiments = np.sum(norm.rvs(size=(m, n)) > 4, axis=1) / n
print(np.mean(experiments))

3.142e-05


In [4]:
std_est = np.sqrt(np.sum((experiments - a) ** 2) / (len(experiments) - 1))
print(2 * std_est / np.sqrt(m))

1.1191632562599122e-06


In [5]:
# importance sampling
# q(z) = exp(-(z - 4)) * 1{z > 4}
# p(z) = N(0, 1)
# w = p(z) / q(z) = exp(z - 4)
# E_p[1{Z > 4}] = E_q[w * 1{Z > 4}] = E_q[w] - E_q[w * 1{Z <= 4}]

# do monte carlo with w
n = int(1e3)
m = 100000
experiments = np.sum(np.exp(norm.rvs(size=(m, n)) - 4) > 1, axis=1) / n
print(np.mean(experiments))

3.1870000000000005e-05


In [6]:
# int(2, +inf)(1 / (pi * (1 + x**2)))
n = int(1e6)
m = 100
X_i = np.random.standard_cauchy(size=(m, n))
experiments = np.mean(X_i > 2, axis=1)
print(np.mean(experiments))

0.14761266


In [18]:
# int(2, +inf)(1 / (pi * (1 + x**2)))
n = int(1e6)
m = 100
X_i = np.random.standard_cauchy(size=(m, n))
experiments = np.mean(np.abs(X_i) > 2, axis=1) / 2
print(np.mean(experiments))

0.14763147


In [19]:
gt = 0.14753257
std_est = np.sqrt(np.sum((experiments - gt) ** 2) / (len(experiments) - 1))
print(std_est)

0.00023152823223795307


In [20]:
n = int(1e7)
U = np.random.uniform(0, 2, size=n)
int_points = 1 / (np.pi + (1 + U ** 2))

print(1 / 2 - np.mean(int_points))

0.3091649122288987


In [21]:
# create pref estimations
aux = 1 / 2 - np.mean(int_points)
pref_ests = (1 / 2 - np.cumsum(int_points) / np.arange(1, n + 1))
std_est = np.sqrt(np.sum((pref_ests - aux) ** 2) / (len(pref_ests) - 1))
print(std_est)

6.698948359082604e-05


In [ ]:
# it should be standard deviations of draws not entire experiments I think

In [ ]:
# double barrier ruin
u = 10 # upper barrier
b = 10 # lower barrier
mi_arr = np.array([0, 0.1, 0.5, 1, 2])
std = 1
n = int(1e5)
ruins = np.zeros((len(mi_arr), n))

for j, mi in enumerate(mi_arr):
    for i in range(n):
        S = 0
        while u > S > -b:
            S += np.random.normal(mi, std)
        if S >= u:
            ruins[j][i] = 1
  
print(np.sum(ruins, axis=1) / n)
print(2 * np.std(ruins, axis=1) / np.sqrt(n))

In [ ]:
# double barrier ruin
u = 10 # upper barrier
b = 10 # lower barrier
# mi_arr = np.array([0, 0.1, 0.5, 1, 2])
mi_arr = np.array([1])
std = 1
n = int(1e7)
ruins = np.zeros((len(mi_arr), n))

for j, mi in enumerate(mi_arr):
    for i in tqdm(range(n)):
        S = 0
        while u > S > -b:
            S += np.random.normal(-mi, std)
        if S >= u:
            ruins[j][i] = np.exp(-2 * mi * S)
  
print(np.sum(ruins, axis=1) / n)
print(2 * np.std(ruins, axis=1) / np.sqrt(n))

In [ ]:
# SAW Self Avoiding Walk, primitive approximation version
d = 2
k = 10
n = int(1e4)

# 2d lattice
lattice = np.zeros((k, k))
lattice[k // 2, k // 2] = 1

# up, right, down, left
directions = np.array([[0, 1], [1, 0], [0, -1], [-1, 0]])

# for each experiment
for _ in range(n):
    # start from the center
    x, y = k // 2, k // 2
    # create a copy of lattice
    lattice_copy = np.copy(lattice)
    # for each step
    for _ in range(k ** 2):
        # shuffle the directions
        np.random.shuffle(directions)
        # for each direction
        for direction in directions:
            # new position
            x_new, y_new = x + direction[0], y + direction[1]
            # if the new position is out of the lattice
            if x_new < 0 or x_new >= k or y_new < 0 or y_new >= k:
                continue
            # if the new position is not visited
            if lattice_copy[x_new, y_new] == 0:
                lattice_copy[x_new, y_new] = 1
                x, y = x_new, y_new
                break
        else:
            # if no direction is available, break
            break

print(np.sum(lattice_copy) / n)

In [ ]:
# SAW Self Avoiding Walk, primitive approximation version
d = 2
k = 10
n = int(2e5)

# 2d lattice
lattice = np.zeros((k, k))
lattice[k // 2, k // 2] = 1

# up, right, down, left
directions = np.array([[0, 1], [1, 0], [0, -1], [-1, 0]])

# for each experiment
saw, n_saw = 0, 0
for _ in range(n):
    # start from the center
    x, y = k // 2, k // 2
    # create a copy of lattice
    lattice_copy = np.copy(lattice)
    # for each step
    for i in range(k):
        # shuffle the directions
        np.random.shuffle(directions)
        # for each direction
        found = False
        for direction in directions:
            # new position
            x_new, y_new = x + direction[0], y + direction[1]
            # if the new position is out of the lattice
            if x_new < 0 or x_new >= k or y_new < 0 or y_new >= k:
                continue
            # if the new position is not visited
            if lattice_copy[x_new, y_new] == 0:
                lattice_copy[x_new, y_new] = 1
                x, y = x_new, y_new
                found = True
                break
        if not found:
            n_saw += 1
            break

        if i == k - 1:
            saw += 1

print(n_saw)
print(saw)
print(saw / n)
print(2**(2*k) * (n_saw / n))